In [ ]:
''' ####label 기억용 개인간 차이 있음
# 0 : 스트레칭 1번 팔 당기기
  1 : 스트레칭 2번 옆구리
  2 : O 표시
  3 : 가만히 있기
  4 : X 표시
  5 : 팔벌려뛰기
  6 : 한 발 서기
  7 : 스쿼트
  8 : 런지
'''

In [3]:
import cv2
import mediapipe as mp
import numpy as np
import time, os

seq_length = 30

# MediaPipe hands model
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

created_time = int(time.time())
os.makedirs('dataset', exist_ok=True)

folder_path='./resource/video/'

video_paths = []
for filename in os.listdir(folder_path):
    if filename.endswith(('.mp4')):  # 비디오 파일 포맷에 따라 확장자를 추가할 수 있음
        video_paths.append(os.path.join(folder_path, filename))


for idx, video_path in enumerate(video_paths):
    cap = cv2.VideoCapture(video_path)
    data = []
    #####
    with mp_pose.Pose(
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            
            ret, image = cap.read()
            if not ret:
                print("카메라를 찾을 수 없습니다.")
                break
            image.flags.writeable = False
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            result = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image,
                result.pose_landmarks,
                mp_pose.POSE_CONNECTIONS,
                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
            if result.pose_landmarks is not None:
                
                joint = np.zeros((33, 4))
                for j, lm in enumerate(result.pose_landmarks.landmark):
                    joint[j] = [lm.x, lm.y, lm.z, lm.visibility]
                    
                # Compute an0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25    
                v1 = joint[[11,11,11,12,12,13,14,15,15,15,16,16,16,17,18,23,23,24,25,26,27,27,28,28,29,30], :3] # Parent joint
                v2 = joint[[12,13,23,14,24,15,16,17,19,21,18,20,22,19,20,24,25,26,27,28,29,31,30,32,31,32], :3] # Child joint
                v = v2 - v1 # [26, 3]
                # Normalize v
                v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]
    
                # Get angle using arcos of dot product
                angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,0,0,0,1,1,3,5,5,5, 6, 6, 6,7,7, 7,8, 8,10,10,10,11,11,15,15,16,17,18,18,19,19,20,20,21,22,22,23 ],:], 
                v[[1,2,3,4,2,5,6,7,8,9,10,11,12,8,9,13,9,13,11,12,14,12,14,16,17,18,19,20,21,22,23,21,24,24,23,25,25 ],:])) 
    
                angle = np.degrees(angle) # Convert radian to degree
    
                angle_label = np.array([angle], dtype=np.float32)
                angle_label = np.append(angle_label, idx)
    
                d = np.concatenate([joint.flatten(), angle_label])
    
                data.append(d)
    
            cv2.imshow('MediaPipe Pose', cv2.flip(image, 1))
            if cv2.waitKey(1) == ord('q'):
                break
        
    data = np.array(data)
    print(idx, data.shape)
    np.save(os.path.join('dataset', f'raw_{idx}_{created_time}'), data)
    
    # Create sequence data
    full_seq_data = []
    for seq in range(len(data) - seq_length):
        full_seq_data.append(data[seq:seq + seq_length])
    
    full_seq_data = np.array(full_seq_data)
    print(idx, full_seq_data.shape)
    np.save(os.path.join('dataset', f'seq_{idx}'), full_seq_data)
    ######


cap.release()
cv2.destroyAllWindows()

I0000 00:00:1719552422.676770  317976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719552422.704115  322745 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719552422.756791  322724 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719552422.767279  322723 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


카메라를 찾을 수 없습니다.
0 (2269, 170)
0 (2239, 30, 170)


I0000 00:00:1719552503.634888  317976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719552503.654178  323021 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719552503.707697  323008 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719552503.721853  323020 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


카메라를 찾을 수 없습니다.
1 (1690, 170)
1 (1660, 30, 170)


I0000 00:00:1719552563.417607  317976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719552563.441183  323235 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719552563.493049  323222 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719552563.507342  323233 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


...

카메라를 찾을 수 없습니다.
6 (1967, 170)
6 (1937, 30, 170)


I0000 00:00:1719552801.070321  317976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719552801.087366  324147 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719552801.137231  324136 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719552801.146624  324145 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


카메라를 찾을 수 없습니다.
7 (1989, 170)
7 (1959, 30, 170)


I0000 00:00:1719552870.571460  317976 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1719552870.590661  324385 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 545.29.06), renderer: NVIDIA GeForce RTX 4060/PCIe/SSE2
W0000 00:00:1719552870.637563  324366 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1719552870.648621  324381 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


카메라를 찾을 수 없습니다.
8 (1849, 170)
8 (1819, 30, 170)
